# Prompting Capabilities 

- Note, you can try any of these prompts outside of this classroom, and without coding, by going to the chat interface [Le Chat](https://chat.mistral.ai/chat).
  - You can sign up with a free account.
  - Signing up for an account is **not** required to complete this course.

In [1]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("MISTRAL_KEY")

- Notice that it's "mistralai", and not "mistral"

### Load API key and helper function
- Note: You can view or download the helper.py file by clicking on the "Jupyter" logo to access the file directory.

In [10]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
mistral = MistralClient(api_key=secret_value_0)
model = "open-mixtral-8x7b"

chat_response = mistral.chat(
    model=model,
    messages=[ChatMessage(role="user", content="What is the best French cheese?")]
)

print(chat_response.choices[0].message.content)

It's impossible to declare a single "best" French cheese, as cheeses vary greatly in taste, texture, and flavor, and preferences for cheese are highly subjective and can depend on the specific context or dish it's being used for.

However, some of the most well-known and highly regarded French cheeses include:

* Brie: a soft, creamy cheese with a white, edible rind.
* Camembert: similar to Brie, but usually stronger in flavor.
* Roquefort: a blue cheese made from sheep's milk, known for its tangy, robust flavor.
* Comté: a hard, nutty cheese made from cow's milk.
* Reblochon: a soft, washed-rind cheese with a strong, earthy flavor.

Ultimately, the best French cheese for you will depend on your personal preferences and the specific dish or occasion you're using it for. It's worth trying a variety of different types to find your favorite!


## Classification

In [11]:
prompt = """
    You are a bank customer service bot. 
    Your task is to assess customer intent and categorize customer 
    inquiry after <<<>>> into one of the following predefined categories:
    
    card arrival
    change pin
    exchange rate
    country support 
    cancel transfer
    charge dispute
    
    If the text doesn't fit into any of the above categories, 
    classify it as:
    customer service
    
    You will only respond with the predefined category. 
    Do not provide explanations or notes. 
    
    ###
    Here are some examples:
    
    Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
    Category: card arrival
    Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
    Category: exchange rate 
    Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
    Category: country support
    Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
    Category: customer service
    ###
    
    <<<
    Inquiry: {inquiry}
    >>>
    Category:
"""

#### Ask Mistral to check the spelling and grammar of your prompt

In [13]:
prompt = f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}"

chat_response = mistral.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

Please correct the spelling and grammar of this prompt and return a text that is the same prompt, with the spelling and grammar fixed:

"You are a bank customer service bot. Your task is to assess customer intent and categorize customer inquiries after <<>>>> into one of the following predefined categories:

* Card arrival
* Change pin
* Exchange rate
* Country support
* Cancel transfer
* Charge dispute

If the text doesn't fit into any of the above categories, classify it as:

* Customer service

You will only respond with the predefined category. Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
Category: Card arrival
Inquiry: I 

#### Try out the model

In [15]:
chat_response = chat_response.choices[0].message.content
prompt = chat_response.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
chat_response = mistral.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

You are a bank customer service bot. Your task is to assess customer intent and categorize customer inquiries after <separator> into one of the following predefined categories:

* Card arrival
* Change pin
* Exchange rate
* Country support
* Cancel transfer
* Charge dispute

If the text does not fit into any of the above categories, classify it as:

* Customer service

You will only respond with the predefined category. Do not provide explanations or notes.

Here are some examples:

Inquiry: "How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?"
Category: Card arrival

Inquiry: "I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any asso

## Information Extraction with JSON Mode

In [16]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker. 
"""

In [21]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema: 

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no"]
        }}
}}
"""

In [22]:
chat_response = mistral.chat(
    model=model,
    response_format={"type": "json_object"},
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

{"age": 60, "gender": "male", "diagnosis": "diabetes", "weight": 210, "smoking": "yes"}


## Personalization

In [31]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year 
fixed rate?

Regards,
Anna
"""

In [32]:
prompt = f"""

You are a mortgage lender customer service bot, and your task is to 
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure 
that your response is clear, concise, and directly addresses the 
customer's question. Address the customer in a friendly and 
professional manner. Sign the email with "Lender Customer Support."   
      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [33]:
chat_response = mistral.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

Dear Anna,

Thank you for reaching out to us regarding our mortgage rates. I'm happy to help you with your inquiry.

For a 30-year fixed-rate mortgage, our current interest rate is 6.403% with an Annual Percentage Rate (APR) of 6.484%. On the other hand, our 15-year fixed-rate mortgage has an interest rate of 5.705% and an APR of 5.848%.

Comparatively, the 15-year fixed-rate mortgage has a lower interest rate and APR than the 30-year fixed-rate mortgage. However, the monthly payments for a 15-year fixed-rate mortgage are typically higher due to the shorter loan term.

I hope this information is helpful to you. If you have any further questions, please don't hesitate to ask.

Best regards,

Lender Customer Support


## Summarization

- We'll use this [article](https://www.deeplearning.ai/the-batch/mistral-enhances-ai-landscape-in-europe-with-microsoft-partnership-and-new-language-models) from The Batch

In [34]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft. 

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context. 

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models. 

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

In [35]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter. 
When presented with the newsletter, come up with interesting questions to ask,
and answer each question. 
Afterward, combine all the information and write a report in the markdown
format. 

# Newsletter: 
{newsletter}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes 
presented in the newsletter.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be 
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed 
in the question.
- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions, 
create a comprehensive report in Markdown format. 
"""

In [37]:
chat_response = mistral.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

# Summary:

European AI champion Mistral AI has unveiled new large language models and formed an alliance with Microsoft. The French startup introduced two new closed models, Mistral Large and Mistral Small, and received a $16.3 million investment from Microsoft. Microsoft will distribute Mistral Large on its Azure platform and collaborate with Mistral AI to train bespoke models for customers, including European governments. Mistral AI's new models outperform some competitors on the MMLU benchmark but fall short of GPT-4. The partnership has divided European lawmakers and regulators, with some expressing concerns about data privacy and antitrust issues.

# Interesting Questions:

Q: How does Mistral AI's performance on the MMLU benchmark compare to other prominent models in the industry?

A: Mistral Large achieved an impressive score of 81.2% on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B. However, it still falls short of GPT-4's 

#### Try it out for yourself
- Feel free to copy-paste text from another article in The Batch, or any other blog or news article.

## The Mistral Python client
- Below is the helper function that you imported from helper.py and used earlier in this notebook.
- For more details, check out the [Mistral AI API documentation](https://docs.mistral.ai/api/)
- To get your own Mistral AI API key to use on your own, outside of this classroom, you can create an account and go to the [console](https://console.mistral.ai/) to subscribe and create an API key.

In [38]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage   

def mistral(user_message, 
            model="mistral-small-latest",
            is_json=False):
    client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            messages=messages)
        
    return chat_response.choices[0].message.content